# Setup

### Import libraries

In [58]:
import os
import random
import collections
from functools import partial

import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
from tqdm import tqdm
import bottleneck as bn

### Set seed

In [59]:
SEED = 42
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
np.random.seed(SEED)
random.seed(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [60]:
# DataLoader에서 셔플사용하고 num_worker가 하나 보다 많을 때, 각각의 워커에 시드 설정을 해주지 않으면 재현성 보장이 안됨
def worker_init_fn(worker_id, seed):
    np.random.seed(seed + worker_id)
    random.seed(seed + worker_id)

worker_init_fn_with_seed = partial(worker_init_fn, seed=SEED)

# Data

### Load file

In [61]:
path = '/data/ephemeral/home/data/cold/'
save_dir = '../saved/'
model_path = save_dir + 'ncf.pth'

# Used to create reindex mapping dicts
full_df = pd.read_csv(path + 'full.csv')
# Used as training data
train_df = pd.read_csv(path + 'train.csv')

# Used for masking during Top K recommendation
val_k = pd.read_csv(path + 'val_k.csv')
test_k = pd.read_csv(path + 'test_k.csv')

# Used as ground truth for Top K recommendation
val_n = pd.read_csv(path + 'val_n.csv')
test_n = pd.read_csv(path + 'test_n.csv')

### Reindex dataframe

In [62]:
users = full_df['user_id'].unique()
items = full_df['item_id'].unique()

user2idx = {user: i for i, user in enumerate(users)}
item2idx = {item: i for i, item in enumerate(items)}

In [63]:
def reindex_df(df, user_mapping, item_mapping):
    df['user_id'] = df['user_id'].map(user_mapping)
    df['item_id'] = df['item_id'].map(item_mapping)
    return

In [64]:
# Reindex dataframes
reindex_df(train_df, user2idx, item2idx)
reindex_df(val_k, user2idx, item2idx)
reindex_df(val_n, user2idx, item2idx)
reindex_df(test_k, user2idx, item2idx)
reindex_df(test_n, user2idx, item2idx)

In [65]:
# full df의 유저 수, 아이템 수
n_users = len(users)
n_items = len(items)

del full_df

In [66]:
# val, test 전체 유저 아이디 리스트
val_users = val_k['user_id'].unique()
test_users = test_k['user_id'].unique()

In [67]:
# 유저 별 인터랙션 개수
val_inter_counts = val_k.groupby('user_id')['item_id'].count()
test_inter_counts = test_k.groupby('user_id')['item_id'].count()

In [68]:
# 1~15-shot 유저 아이디 리스트
val_1_users = val_inter_counts[val_inter_counts == 1].index.tolist()
val_3_users = val_inter_counts[val_inter_counts == 3].index.tolist()
val_5_users = val_inter_counts[val_inter_counts == 5].index.tolist()
val_10_users = val_inter_counts[val_inter_counts == 10].index.tolist()
val_15_users = val_inter_counts[val_inter_counts == 15].index.tolist()

test_1_users = test_inter_counts[test_inter_counts == 1].index.tolist()
test_3_users = test_inter_counts[test_inter_counts == 3].index.tolist()
test_5_users = test_inter_counts[test_inter_counts == 5].index.tolist()
test_10_users = test_inter_counts[test_inter_counts == 10].index.tolist()
test_15_users = test_inter_counts[test_inter_counts == 15].index.tolist()

In [69]:
val_user_groups = {
    'users_1': val_1_users,
    'users_3': val_3_users,
    'users_5': val_5_users,
    'users_10': val_10_users,
    'users_15': val_15_users,
}

test_user_groups = {
    'users_1': test_1_users,
    'users_3': test_3_users,
    'users_5': test_5_users,
    'users_10': test_10_users,
    'users_15': test_15_users,
}

In [70]:
# {user_id: [item_ids]} 딕셔너리 생성, train 에서는 neg_sampling에서 마스킹으로, Top K 추천에서는 scores에 대한 마스킹으로 사용 됌
train_user_train_items = {k: list(v['item_id'].values) for k, v in train_df.groupby('user_id')}
val_user_train_items = {k: list(v['item_id'].values) for k, v in val_k.groupby('user_id')}
test_user_train_items = {k: list(v['item_id'].values) for k, v in test_k.groupby('user_id')}

# val, test 유저들에 대한 정답 아이템 리스트 딕셔너리
val_actual = {k: list(v['item_id'].values) for k, v in val_n.groupby('user_id')}
test_actual = {k: list(v['item_id'].values) for k, v in test_n.groupby('user_id')}

In [71]:
del val_k, test_k, val_n, test_n

### Create COO

In [72]:
def create_coo_matrix(df, n_users, n_items):
    # torch 텐서로 변환
    user_id_tensor = torch.tensor(df['user_id'].values, dtype=torch.long)
    item_id_tensor = torch.tensor(df['item_id'].values, dtype=torch.long)
    label_tensor = torch.ones(len(df), dtype=torch.float32)

    # COO 희소 텐서 생성
    indices = torch.stack([user_id_tensor, item_id_tensor])
    values = label_tensor
    size = (n_users, n_items)  # 전체 유저 x 전체 아이템 크기로 지정

    return torch.sparse_coo_tensor(indices, values, size)

In [73]:
train_coo = create_coo_matrix(train_df, n_users, n_items)

### Dataset Class

In [74]:
pop_values = train_df.groupby('item_id').size().to_numpy()
epsilon = 1e-8
pop_values_log = np.log(pop_values + epsilon)
item_popularity = (pop_values_log - pop_values_log.min()) / (pop_values_log.max() - pop_values_log.min())

In [75]:
del train_df

In [76]:
class BPRDataset(Dataset):
    def __init__(self, coo, user_train_items, item_popularity=None):
        """
        Args:
            user_train_items (dict): 사용자별 상호작용 아이템 딕셔너리.
            num_items (int): 전체 아이템 수.
            n_negs (int): 한 샘플 당 네거티브 샘플 개수.
            item_popularity (list or array): 인덱스에 따른 아이템 인기도 (0~1 범위).
        """
        self.users, self.pos_items = coo._indices()
        self.user_train_items = user_train_items
        self.n_items = coo.shape[1]
        self.n_inter = coo._values().shape[0]
        self.item_popularity = item_popularity
        
        # 아이템 인기도가 주어졌다면, 전역에서 인기도 상위 30%에 해당하는 아이템 리스트를 생성
        if self.item_popularity is not None:
            # 상위 30%에 해당하는 임계값: 70번째 백분위수 이상
            threshold_value = np.percentile(self.item_popularity, 70)
            # 리스트로 변환해서 random.choice가 빠르게 작동하도록 함
            self.top_popular_items = list(set(np.where(np.array(self.item_popularity) >= threshold_value)[0]))
        else:
            self.top_popular_items = None

    def __len__(self):
        return self.n_inter
    
    def __getitem__(self, idx):
        user = self.users[idx]
        pos_item = self.pos_items[idx]

        if self.top_popular_items is not None:
            neg_item = self._get_pop_neg_item(user.item())
        else:  # No top_popular_items_list
            neg_item = self._get_neg_item(user.item())

        return user, pos_item, neg_item
    
    def _get_neg_item(self, user):
        train_items = set(self.user_train_items[user])
        
        neg_item = torch.randint(0, self.n_items, (1,)).item()
        while neg_item in train_items:
            neg_item = torch.randint(0, self.n_items, (1,)).item()
        return neg_item
    
    def _get_pop_neg_item(self, user):
        train_items = set(self.user_train_items[user])

        neg_item = random.choice(self.top_popular_items)
        for _ in range(10):  # 11 번까지만 상위 30퍼 내에서 무작위 샘플링 시도
            if neg_item in train_items:
                neg_item = random.choice(self.top_popular_items)
            else:
                break
        if neg_item in train_items:  # 11번 내에 샘플링 실패 시 전체 아이템에서 무작위 샘플링
            while neg_item in train_items:
                neg_item = torch.randint(0, self.n_items, (1,)).item()
        return neg_item

### DataLoader

In [77]:
train_dataset = BPRDataset(train_coo, train_user_train_items)

In [78]:
train_loader = DataLoader(train_dataset, batch_size=1024, shuffle=True, num_workers=4, worker_init_fn=worker_init_fn_with_seed)

## Model

In [79]:
class NCF(nn.Module):
    def __init__(self, n_users, n_items, emb_dim=64, dropout=0.2):
        super(NCF, self).__init__()
        self.n_users = n_users
        self.n_items = n_items
        self.emb_dim = emb_dim
        self.dropout = dropout

        self.user_emb = nn.Embedding(n_users, emb_dim)
        self.item_emb = nn.Embedding(n_items, emb_dim)
        self.mlp = nn.Sequential(  # [batch_size, emb_dim * 2]
            nn.Dropout(dropout),
            nn.Linear(emb_dim * 2, emb_dim),  # [batch_size, emb_dim]
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(emb_dim, emb_dim // 2),  # [batch_size, emb_dim // 2]
            nn.ReLU(),
            nn.Linear(emb_dim // 2, 1)  # [batch_size, 1]
        )

        self._init_weights()

    def _init_weights(self):
        nn.init.xavier_normal_(self.user_emb.weight)
        nn.init.xavier_normal_(self.item_emb.weight)
        for layer in self.mlp:
            if isinstance(layer, nn.Linear):
                nn.init.kaiming_normal_(layer.weight)
                nn.init.constant_(layer.bias, 0.0)

    def forward(self, user, item):
        user_emb = self.user_emb(user)
        item_emb = self.item_emb(item)
        concat = torch.cat([user_emb, item_emb], dim=-1)  # [batch_size, emb_dim * 2]
        return self.mlp(concat).squeeze()  # [batch_size]

### BPR loss

In [80]:
# BPR 손실 함수
class BPRLoss(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, pos_scores, neg_scores):
        loss = -torch.mean(torch.log(torch.sigmoid(pos_scores - neg_scores)))
        return loss

In [81]:
def recommend_items(model, user_id, num_items, top_k=10, user_train_items=None):
    """
    단일 유저에 대해 전체 아이템에 대한 스코어를 계산한 후,
    이미 학습에 활용된 아이템(train_items)이 있을 경우 이를 마스킹(-무한대로 대체)하고,
    bn.argpartition을 활용해 상위 top_k 아이템을 효율적으로 추출하는 함수.
    
    args:
        model: user_id와 item_id의 텐서를 입력받아 스코어를 반환하는 추천 모델.
        user_id (int): 추천을 위한 대상 유저 ID.
        num_items (int): 전체 아이템의 개수.
        top_k (int): 추천할 아이템 수.
        train_items (list 또는 np.array, optional): 학습 시 활용된 해당 유저의 아이템 인덱스 리스트.
    
    return:
        추천 아이템 인덱스 리스트 (정렬되어 있음)
    """
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    # user_id를 전체 아이템 개수만큼 반복하여 텐서 생성
    user_ids = torch.full((num_items,), user_id, dtype=torch.long).to(device)
    # 모든 아이템의 인덱스 생성
    item_ids = torch.arange(num_items, dtype=torch.long).to(device)
    
    with torch.no_grad():
        scores = model(user_ids, item_ids)
    
    # train_items가 제공되면 해당 아이템의 스코어를 마스킹 처리 (-무한대값으로 대체)
    if user_train_items is not None:
        # torch indexing은 list나 array로도 발동됨
        scores[user_train_items] = -float('inf')
    
    # GPU에 있을 경우 CPU로 옮긴 후 numpy 배열로 변환
    scores_np = scores.cpu().numpy()
    
    # bottleneck의 argpartition을 사용하여 상위 top_k의 후보 인덱스를 추출
    # 음수 부호를 취해 내림차순 정렬 효과를 냄.
    candidate_indices = bn.argpartition(-scores_np, top_k-1)[:top_k]
    
    # argpartition은 정렬되어 있지 않으므로, 위 후보들에 대해 추가 정렬(내림차순) 수행
    sorted_top_indices = candidate_indices[np.argsort(-scores_np[candidate_indices])]
    
    return sorted_top_indices.tolist()

### Metrics

In [82]:
def recall_at_k(actual, predicted, k):
    actual_set = set(actual)
    predicted_at_k = set(predicted[:k])
    return len(actual_set & predicted_at_k) / k

# Same as recall at this experiment environment
# def precision_at_k(actual, predicted, k):
#     actual_set = set(actual)
#     predicted_at_k = set(predicted[:k])
#     return len(actual_set & predicted_at_k) / len(predicted_at_k)

def ndcg_at_k(actual, predicted, k):
    actual_set = set(actual)
    predicted_at_k = predicted[:k]
    dcg = sum([1 / np.log2(i + 2) for i, p in enumerate(predicted_at_k) if p in actual_set])
    idcg = sum([1 / np.log2(i + 2) for i in range(min(len(actual_set), k))])
    return dcg / idcg if idcg > 0 else 0.0

def map_at_k(actual, predicted, k):
    actual_set = set(actual)
    predicted_at_k = predicted[:k]
    score = 0.0
    hit_count = 0.0
    for i, item in enumerate(predicted_at_k):
        if item in actual_set:
            hit_count += 1.0
            score += hit_count / (i + 1)
    return score / len(actual_set) if actual else 0.0


In [83]:
def evaluate(model, users, user_groups, n_items, top_k, user_train_items, user_actual):
    """
    모델의 평가를 위해 전체 아이템에 대한 추천 리스트를 생성하고
    recall@k 등의 평가 지표를 계산하는 함수.
    
    Args:
      model (nn.Module): 추천 스코어를 계산하는 모델.
      users (iterable): 평가할 전체 유저 ID 리스트.
      user_groups (dict): 1,3,5,10,15-shot 유저 ID 리스트 딕셔너리.
      n_items (int): 전체 아이템의 개수.
      top_k (int): 상위 추천 아이템 수 (예: 10).
      user_train_items (dict): 각 유저마다 학습에 사용된 아이템 인덱스 리스트 – 평가 시 해당 아이템은 마스킹 처리함.
      user_actual (dict): 각 유저의 실제 정답(ground truth) 아이템 리스트.
    
    Returns:
      dict: 전체 유저와 1-shot, 3-shot, 5-shot 유저에 대한 평균 recall@k, ndcg@k, map@k 값을 포함하는 딕셔너리.
    """
    model.eval()  # 평가 모드로 전환 (dropout, batchnorm 등 고정)
    all_metrics = {'recall': [], 'ndcg': [], 'map': []}
    metrics_1 = {'recall': [], 'ndcg': [], 'map': []}
    metrics_3 = {'recall': [], 'ndcg': [], 'map': []}
    metrics_5 = {'recall': [], 'ndcg': [], 'map': []}
    metrics_10 = {'recall': [], 'ndcg': [], 'map': []}
    metrics_15 = {'recall': [], 'ndcg': [], 'map': []}
    
    with torch.no_grad():
        for user in tqdm(users, desc="Evaluating", leave=False):
            # 각 유저에 recommend_items 함수 사용 (학습에 사용한 아이템은 마스킹)
            recommendations = recommend_items(model, user, n_items, top_k, user_train_items[user])
            # 추천 결과와 실제 정답(dict나 list)에 따라 메트릭 계산
            recall = recall_at_k(user_actual[user], recommendations, top_k)
            ndcg = ndcg_at_k(user_actual[user], recommendations, top_k)
            map = map_at_k(user_actual[user], recommendations, top_k)

            all_metrics['recall'].append(recall)
            all_metrics['ndcg'].append(ndcg)
            all_metrics['map'].append(map)

            if user in user_groups['users_1']:
                metrics_1['recall'].append(recall)
                metrics_1['ndcg'].append(ndcg)
                metrics_1['map'].append(map)
            elif user in user_groups['users_3']:
                metrics_3['recall'].append(recall)
                metrics_3['ndcg'].append(ndcg)
                metrics_3['map'].append(map)
            elif user in user_groups['users_5']:
                metrics_5['recall'].append(recall)
                metrics_5['ndcg'].append(ndcg)
                metrics_5['map'].append(map)
            elif user in user_groups['users_10']:
                metrics_10['recall'].append(recall)
                metrics_10['ndcg'].append(ndcg)
                metrics_10['map'].append(map)
            elif user in user_groups['users_15']:
                metrics_15['recall'].append(recall)
                metrics_15['ndcg'].append(ndcg)
                metrics_15['map'].append(map)

    model.train()  # 평가 후 다시 학습 모드로 전환

    def average_metrics(metrics):
        return {k: sum(v)/ len(v) for k, v in metrics.items()}
    

    return {
        'all': average_metrics(all_metrics),
        '1-shot': average_metrics(metrics_1),
        '3-shot': average_metrics(metrics_3),
        '5-shot': average_metrics(metrics_5),
        '10-shot': average_metrics(metrics_10),
        '15-shot': average_metrics(metrics_15),
    }


In [84]:
def print_metrics(mode, top_k, metrics):
    """
    주어진 메트릭을 형식에 맞춰 출력하는 함수.
    
    Args:
    - mode (str): 'val', 'best val', 'test' 모드.
    - top_k (int): 상위 추천 아이템 수 (예: 10).
    - metrics (dict): 평가 메트릭 딕셔너리.
    """
    print(f"{mode.capitalize()} - All: Recall@{top_k}: {metrics['all']['recall']:.4f}, "
          f"NDCG@{top_k}: {metrics['all']['ndcg']:.4f}, MAP@{top_k}: {metrics['all']['map']:.4f}")

    print(f"{mode.capitalize()} - 1-shot: Recall@{top_k}: {metrics['1-shot']['recall']:.4f}, "
          f"NDCG@{top_k}: {metrics['1-shot']['ndcg']:.4f}, MAP@{top_k}: {metrics['1-shot']['map']:.4f}")

    print(f"{mode.capitalize()} - 3-shot: Recall@{top_k}: {metrics['3-shot']['recall']:.4f}, "
          f"NDCG@{top_k}: {metrics['3-shot']['ndcg']:.4f}, MAP@{top_k}: {metrics['3-shot']['map']:.4f}")

    print(f"{mode.capitalize()} - 5-shot: Recall@{top_k}: {metrics['5-shot']['recall']:.4f}, "
          f"NDCG@{top_k}: {metrics['5-shot']['ndcg']:.4f}, MAP@{top_k}: {metrics['5-shot']['map']:.4f}")
    
    print(f"{mode.capitalize()} - 10-shot: Recall@{top_k}: {metrics['10-shot']['recall']:.4f}, "
          f"NDCG@{top_k}: {metrics['10-shot']['ndcg']:.4f}, MAP@{top_k}: {metrics['10-shot']['map']:.4f}")
    
    print(f"{mode.capitalize()} - 15-shot: Recall@{top_k}: {metrics['15-shot']['recall']:.4f}, "
          f"NDCG@{top_k}: {metrics['15-shot']['ndcg']:.4f}, MAP@{top_k}: {metrics['15-shot']['map']:.4f}")

### 학습 파라미터 설정

In [90]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
emb_dim = 64
dropout = 0.2
model = NCF(n_users, n_items, emb_dim, dropout).to(device)
criterion = BPRLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0005)
# scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=5)
# optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5)
# scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)

## 학습

In [86]:
epochs = 1
valid_interval = 1
early_stop = 10

top_k = 10

epochs_after_best = 0
best_val_ndcg = None
best_val_metrics = None

for epoch in range(epochs):
    model.train()
    train_loss = 0.0
    for user, pos_item, neg_item in tqdm(train_loader, desc=f'Training Epoch {epoch+1}'):
        user = user.to(device)
        pos_item = pos_item.to(device)
        neg_item = neg_item.to(device)

        pos_scores = model(user, pos_item)
        neg_scores = model(user, neg_item)

        loss = criterion(pos_scores, neg_scores)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
    
    avg_train_loss = train_loss / len(train_loader)
    print(f'Epoch {epoch+1}, Loss: {avg_train_loss:.4f}')

 # 평가: val_users, n_items, top_k, 그리고 유저별 학습/정답 아이템 정보 사용
    if (epoch + 1) % valid_interval == 0:
        # Evaluate on validation users
        val_metrics = evaluate(model, val_users, val_user_groups, n_items, top_k, val_user_train_items, val_actual)
        print_metrics('val', top_k, val_metrics)

        # Check if this is the best validation NDCG
        if best_val_ndcg is None or val_metrics['all']['ndcg'] > best_val_ndcg:
            best_val_ndcg = val_metrics['all']['ndcg']
            best_val_metrics = val_metrics
            best_epoch = epoch + 1

            # Save model
            os.makedirs(save_dir, exist_ok=True)
            torch.save(model.state_dict(), model_path)
            print(f'Current best epoch:{best_epoch}, Model saved to {model_path}')
            epochs_after_best = 0  # Resetting counter
        else:
            epochs_after_best += 1
        if epochs_after_best >= early_stop:
            print(f'Early stopping at epoch {epoch+1}')
            break
    
# Evaluate on test users
model.load_state_dict(torch.load(model_path, weights_only=True))
test_metrics = evaluate(model, test_users, test_user_groups, n_items, top_k, test_user_train_items, test_actual)

# Print best validation metrics
print(f'Best epoch: {best_epoch}')
print_metrics('best val', top_k, best_val_metrics)
# Print test metrics
print(f'Test results from best epoch {best_epoch}')
print_metrics('test', top_k, test_metrics)

Training Epoch 1: 100%|██████████| 12890/12890 [03:35<00:00, 59.69it/s]


Epoch 1, Loss: 0.1105


Val - All: Recall@10: 0.0380, NDCG@10: 0.0675, MAP@10: 0.0339
Val - 1-shot: Recall@10: 0.0297, NDCG@10: 0.0624, MAP@10: 0.0319
Val - 3-shot: Recall@10: 0.0283, NDCG@10: 0.0498, MAP@10: 0.0237
Val - 5-shot: Recall@10: 0.0300, NDCG@10: 0.0509, MAP@10: 0.0243
Val - 10-shot: Recall@10: 0.0472, NDCG@10: 0.0820, MAP@10: 0.0419
Val - 15-shot: Recall@10: 0.0548, NDCG@10: 0.0924, MAP@10: 0.0476
Current best epoch:1, Model saved to ../saved/ncf.pth


Best epoch: 1
Best val - All: Recall@10: 0.0380, NDCG@10: 0.0675, MAP@10: 0.0339
Best val - 1-shot: Recall@10: 0.0297, NDCG@10: 0.0624, MAP@10: 0.0319
Best val - 3-shot: Recall@10: 0.0283, NDCG@10: 0.0498, MAP@10: 0.0237
Best val - 5-shot: Recall@10: 0.0300, NDCG@10: 0.0509, MAP@10: 0.0243
Best val - 10-shot: Recall@10: 0.0472, NDCG@10: 0.0820, MAP@10: 0.0419
Best val - 15-shot: Recall@10: 0.0548, NDCG@10: 0.0924, MAP@10: 0.0476
Test results from best epoch 1
Test - All: Recall@10: 0.0369, NDCG@10: 0.0663, MAP@10: 0.0337
Test - 1-shot: Recall@10: 0.0303, NDCG@10: 0.0651, MAP@10: 0.0342
Test - 3-shot: Recall@10: 0.0285, NDCG@10: 0.0505, MAP@10: 0.0248
Test - 5-shot: Recall@10: 0.0310, NDCG@10: 0.0548, MAP@10: 0.0274
Test - 10-shot: Recall@10: 0.0447, NDCG@10: 0.0774, MAP@10: 0.0400
Test - 15-shot: Recall@10: 0.0499, NDCG@10: 0.0836, MAP@10: 0.0422


## Fine-tune Training

In [88]:
finetune_dataset = BPRDataset(train_coo, train_user_train_items, item_popularity=item_popularity)

In [89]:
finetune_loader = DataLoader(finetune_dataset, batch_size=1024, shuffle=True, num_workers=4, worker_init_fn=worker_init_fn_with_seed)

In [91]:
model.load_state_dict(torch.load('../saved/ncf.pth', weights_only=True))

<All keys matched successfully>

In [ ]:
epochs = 1
valid_interval = 1
early_stop = 10

top_k = 10

epochs_after_best = 0
best_val_ndcg = None
best_val_metrics = None

for epoch in range(epochs):
    model.train()
    train_loss = 0.0
    for user, pos_item, neg_item in tqdm(finetune_loader, desc=f'Training Epoch {epoch+1}'):
        user = user.to(device)
        pos_item = pos_item.to(device)
        neg_item = neg_item.to(device)

        pos_scores = model(user, pos_item)
        neg_scores = model(user, neg_item)

        loss = criterion(pos_scores, neg_scores)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
    
    avg_train_loss = train_loss / len(finetune_loader)
    print(f'Epoch {epoch+1}, Loss: {avg_train_loss:.4f}')

 # 평가: val_users, n_items, top_k, 그리고 유저별 학습/정답 아이템 정보 사용
    if (epoch + 1) % valid_interval == 0:
        # Evaluate on validation users
        val_metrics = evaluate(model, val_users, val_user_groups, n_items, top_k, val_user_train_items, val_actual)
        print_metrics('val', top_k, val_metrics)

        # Check if this is the best validation NDCG
        if best_val_ndcg is None or val_metrics['all']['ndcg'] > best_val_ndcg:
            best_val_ndcg = val_metrics['all']['ndcg']
            best_val_metrics = val_metrics
            best_epoch = epoch + 1

            # Save model
            os.makedirs(save_dir, exist_ok=True)
            torch.save(model.state_dict(), model_path)
            print(f'Current best epoch:{best_epoch}, Model saved to {model_path}')
            epochs_after_best = 0  # Resetting counter
        else:
            epochs_after_best += 1
        if epochs_after_best >= early_stop:
            print(f'Early stopping at epoch {epoch+1}')
            break
    
# Evaluate on test users
model.load_state_dict(torch.load(model_path, weights_only=True))
test_metrics = evaluate(model, test_users, test_user_groups, n_items, top_k, test_user_train_items, test_actual)

# Print best validation metrics
print(f'Best epoch: {best_epoch}')
print_metrics('best val', top_k, best_val_metrics)
# Print test metrics
print(f'Test results from best epoch {best_epoch}')
print_metrics('test', top_k, test_metrics)

Training Epoch 1:   0%|          | 0/12890 [00:00<?, ?it/s]

In [52]:
val_metrics = evaluate(model, val_users, val_user_groups, n_items, top_k, val_user_train_items, val_actual)
print_metrics('val', top_k, val_metrics)
test_metrics = evaluate(model, test_users, test_user_groups, n_items, top_k, test_user_train_items, test_actual)
print_metrics('test', top_k, test_metrics)

Val - All: Recall@10: 0.0000, NDCG@10: 0.0000, MAP@10: 0.0000
Val - 1-shot: Recall@10: 0.0000, NDCG@10: 0.0000, MAP@10: 0.0000
Val - 3-shot: Recall@10: 0.0000, NDCG@10: 0.0000, MAP@10: 0.0000
Val - 5-shot: Recall@10: 0.0000, NDCG@10: 0.0000, MAP@10: 0.0000
Val - 10-shot: Recall@10: 0.0000, NDCG@10: 0.0001, MAP@10: 0.0000
Val - 15-shot: Recall@10: 0.0000, NDCG@10: 0.0000, MAP@10: 0.0000


Test - All: Recall@10: 0.0000, NDCG@10: 0.0000, MAP@10: 0.0000
Test - 1-shot: Recall@10: 0.0000, NDCG@10: 0.0000, MAP@10: 0.0000
Test - 3-shot: Recall@10: 0.0000, NDCG@10: 0.0000, MAP@10: 0.0000
Test - 5-shot: Recall@10: 0.0000, NDCG@10: 0.0000, MAP@10: 0.0000
Test - 10-shot: Recall@10: 0.0000, NDCG@10: 0.0001, MAP@10: 0.0001
Test - 15-shot: Recall@10: 0.0000, NDCG@10: 0.0000, MAP@10: 0.0000


### 테스트 유저에게 추천 및 결과 저장

In [53]:
model.load_state_dict(torch.load('../saved/ncf.pth', weights_only=True))
model.eval()

NCF(
  (user_emb): Embedding(126972, 64)
  (item_emb): Embedding(24790, 64)
  (mlp): Sequential(
    (0): Dropout(p=0.2, inplace=False)
    (1): Linear(in_features=128, out_features=64, bias=True)
    (2): ReLU()
    (3): Dropout(p=0.2, inplace=False)
    (4): Linear(in_features=64, out_features=32, bias=True)
    (5): ReLU()
    (6): Linear(in_features=32, out_features=1, bias=True)
  )
)

In [54]:
# 빈 리스트를 생성하여 추천 결과를 저장
recommendations = []

# 각 사용자에 대해 추천 아이템을 생성하고 리스트에 추가
for user_id in tqdm(test_users):
    recommended_items = recommend_items(model, user_id, n_items, 10, test_user_train_items[user_id])
    for item_id in recommended_items:
        recommendations.append({'user_id': user_id, 'item_id': item_id})

# 리스트를 데이터프레임으로 변환
result = pd.DataFrame(recommendations)

100%|██████████| 12698/12698 [00:16<00:00, 767.97it/s]


In [55]:
idx2user = {v: k for k, v in user2idx.items()}
idx2item = {v: k for k, v in item2idx.items()}

result['user_id'] = result['user_id'].map(idx2user)
result['item_id'] = result['item_id'].map(idx2item)

result.to_csv('../saved/result.csv', index=False)

### 딕셔너리로 추천 결과 생성하는 코드

In [56]:
# test_recommendations = {user_id: recommend_items(model, user_id, n_items, 10, test_user_train_items[user_id]) for user_id in tqdm(test_users)}

In [57]:
# result = pd.DataFrame([
#     {'user_id': user, 'item_id': item}
#     for user, items in test_recommendations.items()
#     for item in items
# ])